In [1]:
import pandas as pd
import numpy as np
import math
from statistics import mode
import random

In [2]:
def distance(a,b):
    distance = math.sqrt(((a-b)**2).sum())
    return distance

In [5]:
def metrics(ts_lb,answer):
    TN = 0
    TP = 0
    FN = 0
    FP = 0
    for i,j in zip(ts_lb,answer):
        if j==1 and i==1:
            TP += 1
        elif(j==1 and i==0):
            FN += 1
        elif(j==0 and i==1):
            FP += 1
        elif(j==0 and i==0):
            TN += 1
    Accuracy = (TP + TN)/(TP + FP + TN + FN)
    Precision = TP/(TP + FP)
    Recall = TP/(TP + FN)
    try:
        f1_score = (2*Precision*Recall)/(Precision + Recall)
    except:
        f1_score = 0
    
    return Accuracy, Precision, Recall, f1_score

In [6]:
def knn(train, test, k):
    answer = []
    ts = test.iloc[:,:-1]
    tr = train.iloc[:,:-1]
    for i in range(len(ts)):
        dist = []
        for j in range(len(tr)):
            distance_a_b = distance((ts.iloc[i]),(tr.iloc[j]))**2
            if distance_a_b !=0:
                dist.append(1/distance_a_b)
            else:
                dist.append(float('inf'))
                
        dist1 = sorted(enumerate(dist), key=lambda x: x[1],reverse = True)
        
        dist1 = dist1[0:k]
        index = [train.iloc[i[0],-1] for i in dist1]
        
        try:
            answer.append(mode(index))
        except :
            answer.append(mode(index[:-1]))
    return answer

    

In [7]:
def k_fold(df):
    nearest = int(input("Enter the K-nearest neighbours: "))
    k = int(input("Enter k fold value: "))
    metrics_list = []
    for i in range(k):
        splitdfs = np.array_split(df, k)
        #print(splitdfs)
        test = splitdfs[i]
        del(splitdfs[i])
        train = pd.concat(splitdfs)
        test.reset_index(inplace = True, drop = True)
        train.reset_index(inplace = True, drop = True) 
        actual = test.iloc[:,-1]
        test = test.iloc[:,:-1]
        
        predicted_values = knn(train, test, nearest)

        Accuracy, Precision, Recall, f1_score = metrics(actual, predicted_values)
        metrics_list.append([Accuracy, Precision, Recall, f1_score])
    metrics_list = np.array(metrics_list)
    metrics_list = np.mean(metrics_list, axis = 0)
    print("Accuracy: ",metrics_list[0])
    print("Precision: ",metrics_list[1])
    print("Recall: ",metrics_list[2])
    print("f1_score: ",metrics_list[3])
    return metrics_list

In [8]:
file_name = 'project3_dataset2.txt'
df = pd.read_csv(file_name, header=None,sep='\t' )

temp = df.iloc[0,:]
category_columns = []
for column in df.columns:
    if type(temp[column]) == str:
        category_columns.append(column)
df.drop(category_columns,axis = 1, inplace=True)
print("Dataset Name: "+file_name)
k_fold(df)

Dataset Name: project3_dataset2.txt
Enter the K-nearest neighbours: 4
Enter k fold value: 3
Accuracy:  0.6168831168831169
Precision:  0.32654127481713685
Recall:  0.426968584251326
f1_score:  0.36957417692988814


array([0.61688312, 0.32654127, 0.42696858, 0.36957418])

# Demo

In [9]:
train = pd.read_csv('project3_dataset3_train.txt', header=None,sep='\t' )
k = int(input("Enter the k nearest neighbour :"))
test = pd.read_csv('project3_dataset3_test.txt', header=None,sep='\t' )

predicted_values = knn(train, test, k)
actual_values = list(test.iloc[:,-1])

Accuracy, Precision, Recall, f1_score = metrics(actual_values ,predicted_values)
print("Accuracy : "+str(Accuracy))
print("Precision : "+str(Precision))
print("Recall : "+str(Recall))
print("f1_score : "+str(f1_score))


Enter the k nearest neighbour :9
Accuracy : 0.95
Precision : 0.9
Recall : 1.0
f1_score : 0.9473684210526316
